In [ ]:
# Cellule 1 Révisée : Installation et Imports

# Utiliser timm (PyTorch Image Models) qui est plus fiable et plus performant
# Si timm n'est pas déjà installé, cette ligne l'installera.
!pip install timm

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import torch.optim as optim
import timm # Nouvel import pour les modèles de vision
import math

# --- Configuration des chemins et constantes ---
DATA_ROOT = '/kaggle/input/csiro-biomass'
TRAIN_IMG_DIR = os.path.join(DATA_ROOT, 'train')
TEST_IMG_DIR = os.path.join(DATA_ROOT, 'test')

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TARGET_NAMES = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
TARGET_WEIGHTS = torch.tensor([0.1, 0.1, 0.1, 0.2, 0.5]).float()

IMAGE_SIZE = 300 
NUM_FOLDS = 5 
BATCH_SIZE = 32
LEARNING_RATE = 1e-4
NUM_EPOCHS = 25

In [ ]:
# Cellule 2 CORRIGÉE : Préparation des Données (Mode Image Seule)

import pandas as pd
import os
from sklearn.model_selection import KFold
# Suppression des imports du préprocesseur (StandardScaler, OneHotEncoder, ColumnTransformer, Pipeline)
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms
from PIL import Image
import numpy as np

# --- A. Chargement des CSV et Conversion du Format Long au Format Large ---
# (Assurez-vous que DATA_ROOT est défini)
train_df_raw = pd.read_csv(os.path.join(DATA_ROOT, 'train.csv'))
test_df_raw = pd.read_csv(os.path.join(DATA_ROOT, 'test.csv'))

# 1. Ajout du Chemin Complet de l'Image
train_df_raw['full_image_path'] = train_df_raw['image_path'].apply(
    lambda x: os.path.join(DATA_ROOT, x)
)
test_df_raw['full_image_path'] = test_df_raw['image_path'].apply(
    lambda x: os.path.join(DATA_ROOT, x)
)

# 2. Conversion du TRAIN au Format Large (Une ligne par image)
train_df_raw['ImageID'] = train_df_raw['sample_id'].apply(lambda x: x.split('__')[0])

# Définir uniquement les colonnes nécessaires (ImageID, chemin, et les cibles)
METADATA_COLS = ['ImageID', 'full_image_path']
train_df_base = train_df_raw[METADATA_COLS].drop_duplicates().reset_index(drop=True)

# Pivoter les cibles
train_df_targets = train_df_raw.pivot(
    index='ImageID', columns='target_name', values='target'
).reset_index()

# Joindre les métadonnées et les cibles
train_df = train_df_base.merge(train_df_targets, on='ImageID', how='left')
TARGET_NAMES = train_df_targets.columns.drop('ImageID').tolist() # Définition des cibles

# 3. Adaptation du TEST au Format Large (Uniquement l'Image et l'ID)
test_df = test_df_raw[['sample_id', 'full_image_path']].drop_duplicates(subset=['full_image_path']).reset_index(drop=True)
test_df['ImageID'] = test_df['sample_id'].apply(lambda x: x.split('__')[0])
        
print(f"✅ DataFrames convertis au format large. TRAIN: {len(train_df)} images. TEST: {len(test_df)} images.")

# --- B. Définition de la Classe PyTorch Dataset (Image Seule) ---

class BiomassDataset(Dataset):
    """
    Dataset pour charger uniquement les images et les cibles (mode Image Seule).
    """
    # L'argument preprocessor a été retiré de l'init
    def __init__(self, df, target_names, transform=None, is_test=False): 
        self.df = df.reset_index(drop=True) 
        self.target_names = target_names
        self.transform = transform
        self.is_test = is_test
        
        if not is_test:
            self.labels = torch.tensor(self.df[self.target_names].values, dtype=torch.float32)
        else:
            self.labels = None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 1. Image
        img_path = self.df.loc[idx, 'full_image_path'] 
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)

        # 2. Rétour
        # Pas de données tabulaires
        if self.is_test:
            return image
        else:
            label = self.labels[idx]
            return image, label

# --- C. Définition des Transformations d'Images (INCHANGÉE) ---

NORM_MEAN = [0.485, 0.456, 0.406]
NORM_STD = [0.229, 0.224, 0.225]
IMAGE_SIZE = 224 # Variable globale nécessaire

# Transformations pour l'entraînement (avec augmentation de données)
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN, std=NORM_STD)
])

# Transformations pour la validation/test (sans augmentation)
valid_test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN, std=NORM_STD)
])

print("✅ Classe BiomassDataset (Image Seule) et transformations définies.")

In [ ]:
# Cellule 3 CORRIGÉE : K-Fold et DataLoaders (Mode Image Seule)

from torch.utils.data import DataLoader
from sklearn.model_selection import KFold

# --- A. Initialisation du K-Fold et du DataLoader de Test ---

# 1. K-Fold pour la Validation Croisée
NUM_FOLDS = 5 # Variable globale nécessaire
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
print(f"✅ Objet KFold créé avec {NUM_FOLDS} plis.")

# 2. DataLoader pour le jeu de Test
# BATCH_SIZE est une variable globale nécessaire ici
if 'BATCH_SIZE' not in locals() and 'BATCH_SIZE' not in globals():
    BATCH_SIZE = 32

test_dataset = BiomassDataset(
    df=test_df, 
    # Plus de préprocesseur à passer en argument
    target_names=TARGET_NAMES, 
    transform=valid_test_transform, 
    is_test=True
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=2, 
    pin_memory=True
)
print(f"✅ DataLoader de Test créé (Taille: {len(test_dataset)}).")

In [ ]:
# Cellule 4 CORRIGÉE : Feature Engineering - Calcul du Pourcentage de Vert et Sec

import cv2
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt # Maintenu pour la visualisation de test

# Nous définissons ici une variable globale pour le masque corrigé afin d'éviter la confusion
MASK_COLOR_MAX = 255 # Valeur d'intensité pour le masque True (blanc)

def calculate_biomass_pct(image_path):
    """
    Calcule le pourcentage de pixels 'verts' (RG-NDVI > 0.1) et 'secs/jaunes' (heuristique).
    Retourne le masque vert en format uint8 pour la compatibilité OpenCV.
    """
    try:
        # Lire l'image en BGR (format OpenCV par défaut)
        img_bgr = cv2.imread(image_path)
        if img_bgr is None:
            return np.nan, np.nan, None 

        # Convertir en float pour les opérations de division
        img_float = img_bgr.astype(float)
        
        # Séparer les canaux B, G, R
        B = img_float[:, :, 0]
        G = img_float[:, :, 1]
        R = img_float[:, :, 2]
        
        # 1. Calcul de l'indice RG-NDVI simplifié 
        epsilon = 1e-6
        rg_ndv = (G - R) / (G + R + epsilon)
        
        # 2. Seuil de Classification
        
        # Masque booléen du VERT (Végétation saine)
        mask_green_bool = rg_ndv > 0.1 
        
        # 🔥 CORRECTION : Convertir le masque booléen en uint8 (0 ou 255) pour OpenCV
        mask_green_uint8 = mask_green_bool.astype(np.uint8) * MASK_COLOR_MAX 

        # Masque du SEC/JAUNE
        mask_dry = (rg_ndv <= 0.1) & (rg_ndv > -0.1) & (R > 50) 
        
        # 3. Calcul des pourcentages
        total_pixels = img_bgr.shape[0] * img_bgr.shape[1]
        
        # Les calculs de pourcentage utilisent la version booléenne car c'est plus direct (True = 1, False = 0)
        pct_green = (np.sum(mask_green_bool) / total_pixels) * 100
        pct_dry = (np.sum(mask_dry) / total_pixels) * 100
        
        # Retourne le masque uint8
        return pct_green, pct_dry, mask_green_uint8

    except Exception as e:
        # print(f"Erreur lors du traitement de {image_path}: {e}")
        return np.nan, np.nan, None

# Mise à jour de la fonction pour l'application (retourne uniquement les features)
def get_biomass_features(image_path):
    # On ignore le masque uint8 lors de l'application aux DataFrames
    pct_green, pct_dry, _ = calculate_biomass_pct(image_path) 
    return pd.Series([pct_green, pct_dry], index=['Pct_Green', 'Pct_Dry'])

print("✅ Fonction de calcul de % Vert / % Sec (RG-NDVI) définie et corrigée.")

# --- A. Test de Visualisation pour Vérifier le Filtre (Utilisation de full_image_path) ---
if len(train_df) > 0:
    example_path = train_df.loc[0, 'full_image_path'] # Utiliser le chemin complet
    
    # pct_green_example, pct_dry_example, mask_green_uint8
    pct_green_example, pct_dry_example, mask_green_uint8 = calculate_biomass_pct(example_path)
    
    # Utiliser le masque uint8 pour la visualisation
    if mask_green_uint8 is not None: 
        img_bgr = cv2.imread(example_path)
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        
        print(f"\nPourcentage de vert (RG-NDVI) pour l'image exemple : {pct_green_example:.2f}%")
        print(f"Pourcentage de sec estimé : {pct_dry_example:.2f}%")
        
        # Affichage des résultats
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.imshow(img_rgb)
        plt.title("Image Originale")
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        # Affichage du masque (uint8 est le bon format pour matplotlib aussi)
        plt.imshow(mask_green_uint8, cmap='gray')
        plt.title(f"Masque Binaire du Vert ({pct_green_example:.2f}%)")
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        # 🔥 CORRECTION APPLIQUÉE ICI : Utilisation de mask=mask_green_uint8 🔥
        masked_image = cv2.bitwise_and(img_bgr, img_bgr, mask=mask_green_uint8)
        masked_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB)
        plt.imshow(masked_image_rgb)
        plt.title("Vert Isolé")
        plt.axis('off')
        
        plt.show()

# --- B. Application du Calcul à l'Ensemble du Dataset ---

print("\nCalcul des pourcentages de biomasse pour le set d'entraînement...")
# Utiliser la fonction apply avec la décomposition en séries pour créer deux colonnes
train_df[['Pct_Green', 'Pct_Dry']] = train_df['full_image_path'].apply(get_biomass_features)

print("Calcul des pourcentages de biomasse pour le set de test...")
test_df[['Pct_Green', 'Pct_Dry']] = test_df['full_image_path'].apply(get_biomass_features)

# 3. Nettoyer les NaN restants (Imputation par la moyenne)
mean_pct_green = train_df['Pct_Green'].mean()
mean_pct_dry = train_df['Pct_Dry'].mean()

train_df['Pct_Green'] = train_df['Pct_Green'].fillna(mean_pct_green)
train_df['Pct_Dry'] = train_df['Pct_Dry'].fillna(mean_pct_dry)

test_df['Pct_Green'] = test_df['Pct_Green'].fillna(mean_pct_green)
test_df['Pct_Dry'] = test_df['Pct_Dry'].fillna(mean_pct_dry)


print(f"✅ Colonnes 'Pct_Green' et 'Pct_Dry' ajoutées aux DataFrames.")
print(f"Moyenne % Vert (Train): {mean_pct_green:.2f}%. Moyenne % Sec (Train): {mean_pct_dry:.2f}%")
print("\nAperçu des nouvelles features (Train) :")
print(train_df[['Pct_Green', 'Pct_Dry'] + TARGET_NAMES].head())

In [ ]:
# Cellule 5 : Définition du Modèle CNN Pur Multi-Cibles avec poids locaux

import torch
import torch.nn as nn
import timm

class CNNBiomassModel(nn.Module):
    """
    Modèle CNN pour régression multi-cibles.
    Permet de charger des poids pré-entraînés locaux pour ResNet-18.
    
    Correction : Le chargement des poids filtre les clés de la couche fc
    pour correspondre à la structure sans tête de classification (num_classes=0).
    """
    def __init__(self, num_targets, model_name='resnet18', pretrained_path=None):
        super().__init__()
        
        # --- A. Modèle de Vision ---
        # Charger le modèle sans la dernière couche (num_classes=0) pour utiliser les features
        self.cnn_model = timm.create_model(model_name, pretrained=False, num_classes=0)
        
        # Si un chemin de poids est fourni, charger les poids locaux
        if pretrained_path is not None:
            # 1. Charger le dictionnaire d'état complet
            state_dict = torch.load(pretrained_path, map_location='cpu')
            
            # 2. Définir les clés à retirer (ce sont les poids de la couche fc standard de ResNet)
            keys_to_remove = ['fc.weight', 'fc.bias']
            
            # 3. Filtrer les poids pré-entraînés pour exclure les clés problématiques
            pretrained_dict_filtered = {k: v for k, v in state_dict.items() if k not in keys_to_remove}
            
            # 4. Charger les poids filtrés dans le modèle
            #    Utiliser strict=False pour ignorer toute autre clé manquante ou inattendue,
            #    tout en chargeant la majorité des poids du ResNet.
            missing_keys, unexpected_keys = self.cnn_model.load_state_dict(pretrained_dict_filtered, strict=False)
            
            if len(unexpected_keys) > 0:
                 print(f"⚠️ Avertissement : {len(unexpected_keys)} clés inattendues ont été ignorées.")
            
            print(f"✅ Poids pré-entraînés chargés avec succès (couche fc exclue) depuis : {pretrained_path}")
        
        # Dimension du vecteur de features (sortie de ResNet après la couche d'AvgPool globale)
        cnn_output_dim = self.cnn_model.num_features
        
        # --- B. Tête de Prédiction (pour votre tâche de Régression Multi-Cibles) ---
        self.prediction_head = nn.Sequential(
            nn.Linear(cnn_output_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_targets)
        )
        
    def forward(self, images):
        # Passage des images dans le CNN (jusqu'à la couche d'AvgPool)
        features = self.cnn_model(images)
        # Passage des features dans la tête de prédiction
        output = self.prediction_head(features)
        return output

# --- Vérification du modèle ---
# Assurez-vous que TARGET_NAMES est défini dans une cellule précédente (sinon l'erreur NameError est levée)
if 'TARGET_NAMES' not in globals():
    # Définition de substitution si nécessaire pour le test (à enlever pour l'environnement Kaggle)
    # TARGET_NAMES = ['target1', 'target2'] 
    raise NameError("TARGET_NAMES doit être définie (liste des noms de vos cibles de régression).")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialisation du modèle avec les poids locaux
model = CNNBiomassModel(
    num_targets=len(TARGET_NAMES),
    model_name='resnet18',
    # Vérifiez que ce chemin est correct dans votre environnement Kaggle
    pretrained_path='/kaggle/input/restnet-weight/resnet18-f37072fd.pth' 
).to(DEVICE)

print(model)

# Vérification rapide avec un batch (nécessite que 'test_loader' soit défini)
try:
    sample_images = next(iter(test_loader))
    if isinstance(sample_images, (list, tuple)):
        # Assurez-vous de prendre le tensor d'images si le loader retourne (image, target)
        sample_images = sample_images[0] 
    sample_images = sample_images.to(DEVICE)
    sample_preds = model(sample_images)
    print("Shape des prédictions :", sample_preds.shape)
except NameError:
    print("⚠️ Attention : 'test_loader' n'est pas défini. Impossible d'effectuer la vérification rapide.")

In [ ]:
# Cellule 6 : Entraînement et Validation K-Fold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import mean_squared_error
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import time
import timm 
# Assurez-vous que CNNBiomassModel est définie (depuis Cellule 5)
# Assurez-vous que train_df, kf, TARGET_NAMES, train_transform, valid_test_transform, DEVICE, BATCH_SIZE sont définis

# --- Hyperparamètres d'Entraînement ---
EPOCHS = 15              
LEARNING_RATE = 1e-4     
MODEL_NAME = 'resnet18' 
# 💥 AJOUT : Chemin vers le poids pré-entraîné local 💥
LOCAL_PRETRAINED_PATH = '/kaggle/input/restnet-weight/resnet18-f37072fd.pth'


# --- 1. Fonction de Perte (Loss Function) ---
loss_fn = nn.MSELoss() 

def calculate_rmse(y_true, y_pred):
    """Calcule la Root Mean Squared Error (RMSE) pour l'ensemble des cibles."""
    if torch.is_tensor(y_true):
        y_true = y_true.detach().cpu().numpy()
        y_pred = y_pred.detach().cpu().numpy()
        
    rms = np.sqrt(mean_squared_error(y_true, y_pred))
    return rms


# --- 2. Boucle Principale K-Fold ---

OOF_PREDS = np.zeros((len(train_df), len(TARGET_NAMES))) # Stockage des prédictions Out-Of-Fold
oof_indices = train_df.index.values 
cv_results = {} 

print(f"Démarrage de la Cross-Validation K-Fold ({kf.n_splits} plis) sur {len(TARGET_NAMES)} cibles...")

# Boucle sur les plis
for fold, (train_idx, val_idx) in enumerate(kf.split(train_df)):
    print(f"\n{'='*20} PLI {fold+1}/{kf.n_splits} {'='*20}")
    start_time = time.time()
    
    # a. Création des DataSets et DataLoaders pour le pli
    full_dataset = BiomassDataset(
        df=train_df,
        target_names=TARGET_NAMES,
        transform=None, 
        is_test=False
    )
    
    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)
    
    # Application des transformations (gestion délicate, on suppose que ça fonctionne)
    train_subset.dataset.transform = train_transform
    val_subset.dataset.transform = valid_test_transform
    
    train_loader = DataLoader(
        train_subset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_subset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    # b. Initialisation du Modèle, de l'Optimiseur et du Scheduler
    # 💥 CORRECTION : Utiliser pretrained_path au lieu de pretrained=False 💥
    model = CNNBiomassModel(
        num_targets=len(TARGET_NAMES), 
        model_name=MODEL_NAME,
        pretrained_path=LOCAL_PRETRAINED_PATH # <--- Correctif pour charger les poids locaux
    ).to(DEVICE)
    
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    
    best_val_rmse = float('inf')
    
    # c. Boucle d'Entraînement
    for epoch in range(EPOCHS):
        # --- PHASE D'ENTRAÎNEMENT ---
        model.train()
        train_loss_sum = 0
        
        for images, targets in train_loader: 
            images, targets = images.to(DEVICE), targets.to(DEVICE)
            
            optimizer.zero_grad()
            
            predictions = model(images) 
            
            loss = loss_fn(predictions, targets)
            loss.backward()
            optimizer.step()
            
            train_loss_sum += loss.item() * images.size(0)
            
        train_loss = train_loss_sum / len(train_subset)
        
        # --- PHASE DE VALIDATION ---
        model.eval()
        val_loss_sum = 0
        all_val_targets = []
        all_val_preds = []
        
        with torch.no_grad():
            for images, targets in val_loader:
                images, targets = images.to(DEVICE), targets.to(DEVICE)
                
                predictions = model(images)
                loss = loss_fn(predictions, targets)
                
                val_loss_sum += loss.item() * images.size(0)
                all_val_targets.append(targets)
                all_val_preds.append(predictions)

        val_loss = val_loss_sum / len(val_subset)
        
        # Calcul de la métrique RMSE
        val_rmse = calculate_rmse(
            torch.cat(all_val_targets), 
            torch.cat(all_val_preds)
        )
        
        # Mise à jour du Scheduler
        scheduler.step(val_loss)

        print(f"Époque {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val RMSE: {val_rmse:.4f}")
        
        # Sauvegarde du meilleur modèle
        if val_rmse < best_val_rmse:
            best_val_rmse = val_rmse
            # Sauvegarde de l'état du modèle
            torch.save(model.state_dict(), f"best_cnn_model_fold_{fold+1}.pth")
            print(f"    ➡️ Modèle sauvegardé (RMSE: {best_val_rmse:.4f})")

    # d. Enregistrement des Prédictions Out-of-Fold (OOF)
    model.load_state_dict(torch.load(f"best_cnn_model_fold_{fold+1}.pth"))
    model.eval()
    
    val_preds_list = []
    with torch.no_grad():
        for images, _ in val_loader:
            images = images.to(DEVICE)
            preds = model(images).cpu().numpy()
            val_preds_list.append(preds)
            
    val_preds = np.concatenate(val_preds_list, axis=0)
    
    # Stockage dans le tableau OOF global
    OOF_PREDS[val_idx, :] = val_preds
    cv_results[f'fold_{fold+1}'] = best_val_rmse
    
    print(f"Durée du pli {fold+1}: {time.time() - start_time:.2f}s")


# --- 3. Résultats Finaux ---

final_rmse = calculate_rmse(train_df[TARGET_NAMES].values, OOF_PREDS)

print(f"\n{'='*50}")
print("✅ Entraînement K-Fold terminé.")
print(f"Résultats par pli (RMSE): {cv_results}")
print(f"SCORE FINAL CV (RMSE OOF): {final_rmse:.4f}")
print(f"{'='*50}")

In [ ]:
# Cellule 7 : Extraction de Features CNN et Modèle k-NN de Stacking

from sklearn.neighbors import KNeighborsRegressor
from torch.utils.data import DataLoader, Dataset
import torch
import numpy as np
import os
import pandas as pd
import timm
from torch import nn 


# --- Variables Globales pour la Cellule ---
# (Assurées d'être définies dans les cellules précédentes)
MODEL_NAME = 'resnet18' 
BATCH_SIZE = 32 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
# Facteur de pondération pour donner plus d'importance au Pct_Green
GREEN_WEIGHT_FACTOR = 2.0 


# --- AJOUT CRITIQUE : Assurer l'existence du Scaler des Cibles (TARGET_MEANS / TARGET_STDS) ---
# Ceci corrige la NameError 'TARGET_MEANS' is not defined.
if 'TARGET_MEANS' not in globals() or 'TARGET_STDS' not in globals():
    try:
        # Re-calcul des moyennes et écarts-types exacts pour la standardisation
        TARGET_MEANS = train_df[TARGET_NAMES].mean().values.astype(np.float32)
        TARGET_STDS = train_df[TARGET_NAMES].std().values.astype(np.float32)
        print("✅ Scaler des cibles recalculé (TARGET_MEANS, TARGET_STDS).")
    except Exception as e:
        # Ceci se déclenchera si train_df ou TARGET_NAMES n'est pas accessible
        raise Exception(f"🛑 ERREUR : Impossible de calculer TARGET_MEANS/STDS. Vérifiez l'exécution des cellules 2 et 5. Erreur: {e}")


# --- 1. Définition du Modèle d'Extraction de Features ---
# (Correct)
class FeatureExtractor(nn.Module):
    """ Modèle qui utilise le backbone du CNN pour renvoyer le feature vector """
    def __init__(self, cnn_model_name):
        super().__init__()
        # Le modèle est créé sans tête de classification et sans poids pré-entraînés (ils seront chargés plus tard)
        self.cnn_model = timm.create_model(cnn_model_name, pretrained=False, num_classes=0)

    def forward(self, x):
        return self.cnn_model(x)

# --- 2. Fonction d'Extraction Générique (CORRIGÉE) ---

def extract_features(df, transform, model_path):
    """Charge le modèle, le met en mode évaluation et extrait les features pour un DataFrame."""
    
    feature_model = FeatureExtractor(MODEL_NAME).to(DEVICE)
    
    # a. Chargement et FILTRAGE des poids du meilleur modèle
    try:
        # 1. Charger le dictionnaire de poids complet
        state_dict = torch.load(model_path, map_location=DEVICE)

        # 2. Filtrer UNIQUEMENT la tête de prédiction. 
        filtered_state_dict = {
            k: v for k, v in state_dict.items() 
            if not k.startswith('prediction_head.')
        }
        
        # 3. Charger le dictionnaire de poids filtré
        feature_model.load_state_dict(filtered_state_dict, strict=False)
        
    except FileNotFoundError:
        print(f"ATTENTION: Fichier modèle non trouvé à {model_path}. Extraction avec poids non entraînés.")
    except Exception as e:
        # Si une autre erreur se produit, on réessaie en mode non strict au cas où
        print(f"⚠️ Erreur de chargement des poids ({e}). Tentative de chargement en mode non strict (strict=False) appliquée.")
        try:
            state_dict = torch.load(model_path, map_location=DEVICE)
            filtered_state_dict = {k: v for k, v in state_dict.items() if not k.startswith('prediction_head.')}
            feature_model.load_state_dict(filtered_state_dict, strict=False)
        except Exception as inner_e:
            raise RuntimeError(f"Échec critique du chargement des features CNN : {inner_e}")
        
    feature_model.eval()
    
    # b. Création du DataLoader (le reste est inchangé)
    dataset = BiomassDataset(
        df=df,
        target_names=TARGET_NAMES,
        transform=transform, 
        is_test=True 
    )
    loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    all_features = []
    with torch.no_grad():
        for images in loader:
            if isinstance(images, (list, tuple)):
                 images = images[0]
                 
            images = images.to(DEVICE)
            features = feature_model(images)
            all_features.append(features.cpu().numpy())
            
    return np.concatenate(all_features, axis=0)

# -------------------------------------------------------------------------------------------------

# --- 3. Exécution de l'Extraction (Inchangée) ---

BEST_MODEL_PATH = f"best_cnn_model_fold_1.pth" 

print(f"⏳ Extraction des features CNN du modèle : {BEST_MODEL_PATH}...")
train_cnn_features = extract_features(train_df, valid_test_transform, BEST_MODEL_PATH)
test_cnn_features = extract_features(test_df, valid_test_transform, BEST_MODEL_PATH)

print(f"✅ Features CNN TRAIN extraites. Shape: {train_cnn_features.shape}")
print(f"✅ Features CNN TEST extraites. Shape: {test_cnn_features.shape}")


# -------------------------------------------------------------------------------------------------

# --- 4. Combinaison des Features (CNN + Manuelles PONDÉRÉES) ---

manual_features_cols = ['Pct_Green', 'Pct_Dry'] 
train_manual_features = train_df[manual_features_cols].copy()
test_manual_features = test_df[manual_features_cols].copy()

# 🔥 PONDÉRATION DU VERT 🔥
print(f"\nAppliquer la pondération GREEN_WEIGHT_FACTOR ({GREEN_WEIGHT_FACTOR:.1f}) sur Pct_Green...")
train_manual_features['Pct_Green'] *= GREEN_WEIGHT_FACTOR
test_manual_features['Pct_Green'] *= GREEN_WEIGHT_FACTOR

X_train_manual = train_manual_features.values
X_test_manual = test_manual_features.values

# Concaténation des Features
X_train_final = np.concatenate([train_cnn_features, X_train_manual], axis=1)
X_test_final = np.concatenate([test_cnn_features, X_test_manual], axis=1)

# 💥 CORRECTION 2 : Standardisation des Cibles 💥
Y_train_raw = train_df[TARGET_NAMES].values
Y_train_scaled = (Y_train_raw - TARGET_MEANS) / TARGET_STDS 

print(f"\nShape finale du Feature Set TRAIN : {X_train_final.shape}")
print(f"Shape finale du Feature Set TEST : {X_test_final.shape}")


# -------------------------------------------------------------------------------------------------

# --- 5. Modélisation k-NN (Régression Multi-Cibles) ---

N_NEIGHBORS = 15 
print(f"\n⏳ Entraînement du KNeighborsRegressor (k={N_NEIGHBORS}) sur le feature set combiné...")

knn_model = KNeighborsRegressor(
    n_neighbors=N_NEIGHBORS, 
    weights='distance', 
    metric='euclidean'
)

# Utilisation des cibles STANDARDISÉES pour l'entraînement du k-NN
knn_model.fit(X_train_final, Y_train_scaled)

print("✅ Modèle k-NN entraîné.")


# --- 6. Prédictions Finales ---
# Les prédictions ici seront des valeurs STANDARDISÉES !
final_predictions_knn = knn_model.predict(X_test_final)

print(f"\nShape des prédictions finales (TEST) : {final_predictions_knn.shape}")

In [ ]:
# Cellule 8 CORRIGÉE (Finale) : Dés-standardisation, Formatage et Soumission du Modèle k-NN

import pandas as pd
import numpy as np

# Les variables suivantes doivent être définies à partir des cellules précédentes :
# - final_predictions_knn (prédictions finales du k-NN - Cellule 7, VALEURS STANDARDISÉES)
# - test_df (DataFrame de test - Cellule 2)
# - TARGET_NAMES (liste des cibles - Cellule 2)
# - TARGET_MEANS et TARGET_STDS (scalers - Cellule 7)

# --- 1. DÉS-STANDARDISATION des Prédictions ---
print(f"⏳ Dés-standardisation des {final_predictions_knn.shape[0]} prédictions...")
final_predictions_real = (final_predictions_knn * TARGET_STDS) + TARGET_MEANS
print("✅ Dés-standardisation terminée.")


# --- 2. Formatage et Application des Contraintes ---

final_preds_list = []
# CORRECTION MAJEURE : Remplacer 'ID' par 'ImageID' pour éviter la KeyError
test_sample_ids = test_df['ImageID'].values 

print(f"⏳ Formatage des prédictions pour la soumission...")

# Boucle principale
for i in range(final_predictions_real.shape[0]):
    # Utilisation de l'ID pour la construction du sample_id
    image_id_prefix = test_sample_ids[i]

    for j, target_name in enumerate(TARGET_NAMES):
        # Récupération de la valeur prédite (maintenant dés-standardisée)
        value = final_predictions_real[i][j]
        
        # a) Contrainte Compétition (>= 0.5) 
        # Application de la règle sauvegardée : "Le minimum de score requis... est de 0,5."
        value = max(0.5, value) 
        
        # Le sample_id final doit être au format {ImageID}__{target_name}
        submission_sample_id = f"{image_id_prefix}__{target_name}"
        
        final_preds_list.append({
            "sample_id": submission_sample_id,
            "target": value
        })

# --- 3. Création du Fichier CSV ---
submission_df = pd.DataFrame(final_preds_list)

# CORRECTION CRUCIALE : Utilisation du nom de fichier requis par la compétition
submission_path = "submission.csv" 
submission_df.to_csv(submission_path, index=False)

print(f"\n✅ Soumission générée (Nom de Fichier et ID Corrigés) : {submission_path} ({len(submission_df)} lignes)")
print("Aperçu de la soumission :")
print(submission_df.head(5))